In [1]:
import ssid
import quakeio
from ssid import modal
from utilities import Config, extract_channels, list_files, print_modes, mode_statistics

### Data inputs

In [2]:
directory = "uploads/painter"
pattern  = "*.[zZ][iI][pP]"

In [3]:
data_conf = Config()
data_conf.d = 8 # decimation factor

# PAINTER RIO DELL TRANSVERSE (CASE 1)
data_conf.inputs  = [17,3,20]
data_conf.outputs = [9,7,4]
# # PAINTER RIO DELL LONGITUDINAL (CASE 2)
# data_conf.inputs  = [15,1,18]
# data_conf.outputs = [11]

# # HWY8/MELOLAND TRANSVERSE (CASE 1)
# data_conf.inputs  = [2],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND TRANSVERSE (CASE 2)
# data_conf.inputs  = [11,2,26],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND LONGITUDINAL (CASE 3)
# data_conf.inputs  = [12,4,25],
# data_conf.outputs = [27,8]

# # CROWLEY TRANSVERSE (CASE 1)
# data_conf.inputs = [4]
# data_conf.outputs = [6,7,9]
# # CROWLEY TRANSVERSE (CASE 2)
# data_conf.inputs = [6,4,9]
# data_conf.outputs = [7]
# # CROWLEY TRANSVERSE (CASE 3)
# data_conf.inputs = [4]
# data_conf.outputs = [7]
# # CROWLEY LONGITUDINAL (CASE 4)
# data_conf.inputs = [5]
# data_conf.outputs = [8]

# # RIDGECREST TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [6,7,9]

# # CAPISTRANO TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [10,7]

# # HAYWARD TRANSVERSE (CASE 1)
# data_conf.inputs  = [2,7,25,18]
# data_conf.outputs = [13,15,23,20]
# # HAYWARD LONGITUDINAL
# data_conf.inputs  = [3,6,17],
# data_conf.outputs = [12,14,19]

# # BERNARDINO TRANSVERSE BENT 3 (CASE 1)
# data_conf.inputs  = [6]
# data_conf.outputs = [7,8]
# # BERNARDINO TRANSVERSE BENT 8 (CASE 2)
# data_conf.inputs  = [24]
# data_conf.outputs = [19,20]
# # BERNARDINO LONGITUDINAL BENT 3 (CASE 3)
# data_conf.inputs  = [4]
# data_conf.outputs = [10]
# # BERNARDINO LONGITUDINAL BENT 8 (CASE 4)
# data_conf.inputs  = [22]
# data_conf.outputs = [17,18]


### Method Inputs

### General Parameters
parameter | value
----------|-------
`p ` | number of output channels
`q ` | number of input channels
`nt` | number of timesteps
`dt` | timestep
`d`  | decimation (downsampling) factor

### Specific to Observer Kalman Identification (OKID)
parameter | value
----------|-------
`m`  | number of Markov parameters to compute (at most = nt)

### Specific to Eigensystem Realization Algorithm (ERA)
parameter | value
----------|-------
`no` | number of observability parameters
`nc` | number of controllability parameters
`r ` | reduced model order

### Specific to Data Correlations (DC)
parameter | value
----------|-------
`a`  | (alpha) number of additional block rows in Hankel matrix of correlation matrices
`b`  | (beta) number of additional block columns in Hankel matrix of correlation matrices
`l`  | initial lag
`g`  | lag (gap) between correlations

### Specific to System Realization with Information Matrix (SRIM)
parameter | value
----------|-------
`no` | number of steps used for identification (prediction horizon)

In [4]:
# Set Parameters
conf = Config()
conf.m  = 210
conf.no = 100
conf.nc = 100
conf.r  = 12
conf.a  = 0  
conf.b  = 0  
conf.l  = 10 
conf.g  = 3

In [5]:
files = list_files(directory, pattern)
event_modes = []
for file in files:
    print(file)
    try:
        event = quakeio.read(file, exclusions=["*filter*", "*date*"])
        print("peak acceleration (cm/s/s):", event["peak_accel"])
        inputs, dt = extract_channels(event, data_conf["inputs"], decimate=data_conf["d"])
        outpts, dt = extract_channels(event, data_conf["outputs"], decimate=data_conf["d"])
        realization,Observability = ssid.system(method="srim", inputs=inputs, outputs=outpts, **conf)
    except Exception as e:
        print(e)
        continue
    ss_modes = modal.system_modes(realization,dt,nt=inputs.shape[1])
    event_modes.append(list(ss_modes.values()))
    print_modes(ss_modes)

event_frequencies = mode_statistics(event_modes, "freq")
event_frequencies

uploads/painter/bayview_11oct2013_72086051_ce89324p.zip
peak acceleration (cm/s/s): 87.954


100%|█████████▉| 1400/1401 [00:00<00:00, 1448.36it/s]


Spectral quantities:
       T(s)        ζ        EMACO        MPC     EMACO*MPC
      0.2647     -0.00271   1.0        0.9978     0.9978   
      0.2493     -0.004814  1.0        0.9839     0.9839   
      0.234      0.003079   1.0        0.9816     0.9816   
      0.2291     0.05486    0.0        0.9411     0.0      
      0.2239     0.02438    0.0        0.7533     0.0      
      0.207      0.007817   1.0        0.9403     0.9402   
Mean Period(s): 0.23466418540798042
Standard Dev(s): 0.018390282114705357
uploads/painter/CapeMendocinoAftershock_21Nov1986_CE89324P.ZIP
peak acceleration (cm/s/s): 343.36


 97%|█████████▋| 38/39 [00:00<00:00, 18797.45it/s]

Spectral quantities:
       T(s)        ζ        EMACO        MPC     EMACO*MPC
      0.6989     0.8668     0.0        0.4988     0.0      
      0.5157     0.3334     0.0        0.5216     0.0      
      0.4477     0.5711     0.0        0.7179     0.0      
      0.3521     0.1801     0.0        0.8911     0.0      
      0.3437     0.06236    1.0        0.9471     0.9471   
Mean Period(s): 0.4716125302466416
Standard Dev(s): 0.1302357300182834
uploads/painter/CapeMendocino_21Nov1986_CE89324P.ZIP


...

100%|█████████▉| 721/722 [00:00<00:00, 2653.37it/s]


Spectral quantities:
       T(s)        ζ        EMACO        MPC     EMACO*MPC
      0.2451     0.01403    1.0        0.9567     0.9567   
      0.2306     0.01718    0.9997     0.9974     0.9971   
      0.228      0.003574   1.0        0.9789     0.9789   
      0.1901     0.009443   1.0        0.9935     0.9935   
      0.1673     0.00475    1.0        0.9749     0.9749   
      0.1632     0.002132   1.0        0.9267     0.9267   
Mean Period(s): 0.20402535777743638
Standard Dev(s): 0.032077922473309914
uploads/painter/Trinidad-Offshore_08Nov1980_CE89324p.ZIP
peak acceleration (cm/s/s): -326.247


Only extracted 1 channels, 2 missing.
 97%|█████████▋| 38/39 [00:00<00:00, 29993.14it/s]

Spectral quantities:
       T(s)        ζ        EMACO        MPC     EMACO*MPC
      0.5037     0.3051     0.0        1.0        0.0      
      0.4376     0.08908    1.0        1.0        1.0      
      0.4191     0.3924     8.053e-29  1.0        8.053e-29
      0.3289     0.2036     0.0        1.0        0.0      
Mean Period(s): 0.4223397844807927
Standard Dev(s): 0.06243795472282476


[{'freq': 4.011802053113038,
  'damp': -0.004813843306062524,
  'modeshape': array([-0.00505015+0.03837154j,  0.00012459+0.04997484j,
         -0.00019496+0.01187357j]),
  'cnd': 1.2053813503180704,
  'energy_condensed_emaco': 0.9999999999994221,
  'mpc': 0.9838660376058102,
  'distance': -0.051443052746744024},
 {'freq': 2.9098305467567798,
  'damp': 0.062363497407253415,
  'modeshape': array([0.12751218+0.03323861j, 0.23356202-0.01276542j,
         0.1661325 -0.00830496j]),
  'cnd': 1.211285605782191,
  'energy_condensed_emaco': 0.9999999999997208,
  'mpc': 0.9471196459758631,
  'distance': -1.077224677106074},
 {'freq': 3.035818273922013,
  'damp': 0.06044690636110912,
  'modeshape': array([0.00475533-0.04179163j, 0.10519837+0.07871083j,
         0.08098019+0.06440362j]),
  'cnd': 1.6036897391830875,
  'energy_condensed_emaco': 0.9999999999997912,
  'mpc': 0.8347097699703233,
  'distance': -0.959947692469493},
 {'freq': 4.153023409466451,
  'damp': 0.0073231523480792714,
  'modeshap

### Painter St Rio Dell Hwy 101 89324
![Painter St Rio Dell Hwy 101 89324 Sensors 1](./figures/89324_Page_1.png)
![Painter St Rio Dell Hwy 101 89324 Sensors 2](./figures/89324_Page_2.png)

### Meloland Hwy 8 El Centro 01336
![Meloland Hwy 8 El Centro 01336 Sensors](./figures/01336.png)

### Lake Crowley Hwy 395 54730
![Lake Crowley Hwy 395 54730 Sensors](./figures/54730.png)

### Ridgecrest 33472
![Ridgecrest 33472 Sensors](./figures/33742.png)

### Capistrano 13795
![Capistrano 13795 Sensors](./figures/13795.png)

### Hayward Hwy 580-238 58658
![Hayward Hwy 580-238 58658 Sensors 1](./figures/58658_Page_1.png)
![Hayward Hwy 580-238 58658 Sensors 2](./figures/58658_Page_2.png)

### San Bernardino I10-215 23631
![San Bernardino I10-215 Sensors](./figures/23631.png)